# Lista 10
Część pierwsza sprawozdania.

## Zadanie 1

In [65]:
install.packages("kableExtra")

library(knitr)
library(kableExtra)
library(IRdisplay)

Installing package into 'C:/Users/stryc/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

also installing the dependencies 'rmarkdown', 'scales', 'viridisLite', 'rstudioapi', 'webshot', 'svglite'




package 'rmarkdown' successfully unpacked and MD5 sums checked
package 'scales' successfully unpacked and MD5 sums checked
package 'viridisLite' successfully unpacked and MD5 sums checked
package 'rstudioapi' successfully unpacked and MD5 sums checked
package 'webshot' successfully unpacked and MD5 sums checked
package 'svglite' successfully unpacked and MD5 sums checked
package 'kableExtra' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\stryc\AppData\Local\Temp\RtmpILmsAD\downloaded_packages



Attaching package: 'kableExtra'


The following object is masked from 'package:dplyr':

    group_rows




## Zadanie 2

## Zadanie 3

# Lista 11
Część druga sprawozdania.

## Zadanie 1

W zadaniu pierwszym przeprowadzimy test symetrii, quasi-symetrii i quasi-niezależności dla danych dotyczących ocen uzyskanych przez studentów z dwóch kolokwiów. Wszystkie testy zostaną przeprowadzone na poziomie istotności $0.05$.

In [59]:
# HIDE INPUT

count <- c(5, 2, 1, 0, 0, 0,
           6, 3, 2, 2, 0, 0,
           1, 4, 5, 5, 2, 2,
           0, 10, 15, 18, 5, 2,
           1, 2, 5, 3, 2, 2,
           0, 1, 3, 4, 3, 2)

data <- matrix(count, byrow=TRUE, nrow=6)
rownames(data) = c("2", "3", "+3", "4", "+4", "5")
colnames(data) = c("2", "3", "+3", "4", "+4", "5")

data %>% 
    kable("html") %>%
    as.character() %>%
    display_html()

ERROR: Error in table(count, byrow = TRUE, nrow = 6, rownames = c("2", "3", "+3", : all arguments must have the same length


In [58]:
?matrix

### Symetria
Przy testowaniu symetrii, moglibyśmy skorzystać z testu Bowkera, jednak w przypadku naszych danych nie jest to możliwe. Wywołanie tego testu (`mcnemar.test`) zwróciłoby p-wartość `NaN`. Dlatego należy skorzystać z testu ilorazu wiarygodności, zaimplementowanego w funkcji `glm` biblioteki `gnm`.

In [39]:
#install.packages("gnm")
#install.packages("dplyr")
library(gnm)
library(dplyr)

count <- c(5, 2, 1, 0, 0, 0,
           6, 3, 2, 2, 0, 0,
           1, 4, 5, 5, 2, 2,
           0, 10, 15, 18, 5, 2,
           1, 2, 5, 3, 2, 2,
           0, 1, 3, 4, 3, 2)

K1 <- gl(6,6, labels=c("2", "3", "+3", "4", "+4", "5"))
K2 <- gl(6,1, labels=c("2", "3", "+3", "4", "+4", "5"))
data <- data.frame(K1, K2, count)

symmetry <- glm(count ~ Symm(K1, K2), data=data, family=poisson)
symmetry %>% summary()


Call:
glm(formula = count ~ Symm(K1, K2), family = poisson, data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8988  -0.5463   0.0000   0.4981   1.4888  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        1.609e+00  4.472e-01   3.599  0.00032 ***
Symm(K1, K2)2:3   -2.231e-01  5.701e-01  -0.391  0.69549    
Symm(K1, K2)2:+3  -1.609e+00  8.367e-01  -1.924  0.05440 .  
Symm(K1, K2)2:4   -1.991e+01  4.043e+03  -0.005  0.99607    
Symm(K1, K2)2:+4  -2.303e+00  1.095e+00  -2.102  0.03556 *  
Symm(K1, K2)2:5   -1.991e+01  4.043e+03  -0.005  0.99607    
Symm(K1, K2)3:3   -5.108e-01  7.303e-01  -0.699  0.48425    
Symm(K1, K2)3:+3  -5.108e-01  6.055e-01  -0.844  0.39889    
Symm(K1, K2)3:4    1.823e-01  5.323e-01   0.343  0.73196    
Symm(K1, K2)3:+4  -1.609e+00  8.367e-01  -1.924  0.05440 .  
Symm(K1, K2)3:5   -2.303e+00  1.095e+00  -2.102  0.03556 *  
Symm(K1, K2)+3:+3  2.561e-15  6.325e-01   0.000  1.00000    
Symm

Policzmy zatem p-wartość.

In [45]:
x = symmetry$deviance
r = 15

p = 1 - pchisq(x, r)
p %>% round(4)

[1] 0.1005

Otrzymujemy p-wartość na poziomie $0.1005 < 0.05$, więc odrzucamy hipotezę o symetrii.

### Quasi-symetria
Ponownie skorzystamy z funkcji `glm`.

In [46]:
quasi_symm <- glm(count ~ K1+K2 + Symm(K1,K2), data=data, family =poisson)
summary(quasi_symm)


Call:
glm(formula = count ~ K1 + K2 + Symm(K1, K2), family = poisson, 
    data = data)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.86903  -0.08908   0.00000   0.06047   0.80881  

Coefficients: (5 not defined because of singularities)
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)          1.60944    0.44721   3.599  0.00032 ***
K13                 -1.83252    1.17661  -1.557  0.11936    
K1+3                 0.29184    0.76722   0.380  0.70365    
K14                  1.16686    0.72161   1.617  0.10587    
K1+4                 0.86552    0.77135   1.122  0.26183    
K15                  0.85966    0.63615   1.351  0.17658    
K23                 -2.61466    1.16128  -2.252  0.02435 *  
K2+3                -1.11675    0.74555  -1.498  0.13416    
K24                 -1.28102    0.71818  -1.784  0.07447 .  
K2+4                -1.37441    0.75441  -1.822  0.06848 .  
K25                 -1.77595    0.63615  -2.792  0.00524 ** 

In [51]:
x = quasi_symm$deviance
r = 10
p = 1 - pchisq(x, r)

p %>% round(4)

[1] 0.9589

Otrzymana p-wartość na poziomie $0.9589$ nie daje podstaw do odrzucenia hipotezy o quasi-symetrii.

### Quasi-niezależność
Do testowania quasi-niezależności również możemy wykorzystać funkcję `glm`.

In [53]:
quasi_ind <-  glm(count ~ K1 + K2 + Diag(K1, K2), data=data, family = poisson)
summary(quasi_ind)


Call:
glm(formula = count ~ K1 + K2 + Diag(K1, K2), family = poisson, 
    data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5605  -0.6443  -0.0649   0.2053   3.3845  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -1.3500     0.6876  -1.963 0.049590 *  
K13              1.3547     0.6616   2.048 0.040602 *  
K1+3             1.8306     0.6423   2.850 0.004372 ** 
K14              2.5373     0.6092   4.165 3.12e-05 ***
K1+4             1.5024     0.6425   2.338 0.019370 *  
K15              1.2838     0.6527   1.967 0.049190 *  
K23              0.9628     0.4241   2.270 0.023181 *  
K2+3             1.3646     0.4095   3.333 0.000861 ***
K24              1.0334     0.4545   2.274 0.022994 *  
K2+4             0.3432     0.4768   0.720 0.471570    
K25             -0.1993     0.5417  -0.368 0.712931    
Diag(K1, K2)2    2.9595     0.8202   3.608 0.000308 ***
Diag(K1, K2)3    0.1312     0.7110   0.184 0.853635   

In [55]:
x = quasi_ind$deviance
r = 19
p = 1 - pchisq(x, r)
p %>% round(4)

[1] 0.0096

Test ilorazu wiarygodności zwraca p-wartość $0.0096 < 0.05$, więc odrzucamy hipotezę zerową o quasi-niezależności.

## Zadanie 2

zad 2 
trzeba wskazac ze rozklady brzegowe sa jednorodne

na poprzedniej liscie weryfikowalismy hip ze dane pochodza z symetrii (?)

wynik z testu symetrii jakby jednoznacznie okreslal czy mozemy odrzucic hipoteze o jednorodnosci czy nie

tutaj sytuacja jest bardziej skomplikowana poniewaz symetria nie jest rownowazna jednorodnosci

symetria implikuje jednorodnosc rozkladu (implikacja tylko w 1 strone)

w kontekscie zadania drugiego,
jezeli w podpunkcie a w 1 zadaniu wyjdzie ze nie mozna odrzucic hipotezy, bedziemy wnioskowac ze dane pochodza z modelu symetrii, to automatycznie mowimy ze rozklady brzegowe za jednorodne

jezeli w 1a odrzucamy hipoteze, to nie mozemy automatycznie odrzucic hipotezy

w zadaniu 2 powolujemy sie na wynik z 1a jesli ta nie odrzucimy hipotezy o symetrii

nie bedziemy miec podstaw do odrzucenia wiec mzoemy tak zrobic


drugi test jaki mozna wykonac to test bazujacy na ilorazie wiarygnodnosci

na wykladzie mielismy na kilku slajdach rozpisane ze pod warunkiem jest zachodzi quasi symetria to...

test z wykladu w zadaniu drugim, ostatni podrozdzial kilka linijek

przeczytanie przykladu do wykladu i powtorzenie analogicznych testow dla naszego  przykladu

nie bedzie trudne i pracochlonne